In [1]:
import numpy as np
import pandas as pd
trainsz = 2245
testsz = 2243
path_train_x = '/kaggle/input/up-solving-tst-day-1/train/train/filtered_images/'
path_train_y = '/kaggle/input/up-solving-tst-day-1/train/train/real_images/'
path_test_x = '/kaggle/input/up-solving-tst-day-1/test/test/filtered_images/'

In [2]:
def generate_samples(original_img, distorted_img, window=3):
    H, W, _ = original_img.shape
    pad = window // 2
    X = []
    Y = []
    for i in range(pad, H - pad):
        for j in range(pad, W - pad):
            patch = distorted_img[i-pad:i+pad+1, j-pad:j+pad+1, :].flatten()
            target = original_img[i, j, :]  # Истинные RGB значения
            X.append(patch)
            Y.append(target)
    return np.array(X), np.array(Y)

In [3]:
from PIL import Image
train_images_x = np.zeros((trainsz, 128, 128, 3))
for i in range(trainsz):
    train_images_x[i] = np.array(Image.open(path_train_x + str(i) + ".png"))
    if (i + 1) % (trainsz // 100) == 0:
        print("trainx preprocess " + str((i + 1) / (trainsz // 100)))
train_images_y = np.zeros((trainsz, 128, 128, 3))
for i in range(trainsz):
    train_images_y[i] = np.array(Image.open(path_train_y + str(i) + ".png"))
    if (i + 1) % (trainsz // 100) == 0:
        print("trainy preprocess " + str((i + 1) / (trainsz // 100)))
test_images_x = np.zeros((testsz, 128, 128, 3))
for i in range(testsz):
    test_images_x[i] = np.array(Image.open(path_test_x + str(i) + ".png"))
    if (i + 1) % (testsz // 100) == 0:
        print("test preprocess " + str((i + 1) / (testsz // 100)))

trainx preprocess 1.0
trainx preprocess 2.0
trainx preprocess 3.0
trainx preprocess 4.0
trainx preprocess 5.0
trainx preprocess 6.0
trainx preprocess 7.0
trainx preprocess 8.0
trainx preprocess 9.0
trainx preprocess 10.0
trainx preprocess 11.0
trainx preprocess 12.0
trainx preprocess 13.0
trainx preprocess 14.0
trainx preprocess 15.0
trainx preprocess 16.0
trainx preprocess 17.0
trainx preprocess 18.0
trainx preprocess 19.0
trainx preprocess 20.0
trainx preprocess 21.0
trainx preprocess 22.0
trainx preprocess 23.0
trainx preprocess 24.0
trainx preprocess 25.0
trainx preprocess 26.0
trainx preprocess 27.0
trainx preprocess 28.0
trainx preprocess 29.0
trainx preprocess 30.0
trainx preprocess 31.0
trainx preprocess 32.0
trainx preprocess 33.0
trainx preprocess 34.0
trainx preprocess 35.0
trainx preprocess 36.0
trainx preprocess 37.0
trainx preprocess 38.0
trainx preprocess 39.0
trainx preprocess 40.0
trainx preprocess 41.0
trainx preprocess 42.0
trainx preprocess 43.0
trainx preprocess 44

In [4]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
from skimage.color import rgb2hsv, rgb2lab
from skimage.filters import sobel
from tqdm import tqdm

def extract_enhanced_features(imgs_x, imgs_y, patch_size=5, samples_per_image=1000):
    pad = patch_size // 2
    features = []
    targets = []
    
    for i in tqdm(range(imgs_x.shape[0]), desc="Extracting enhanced patches"):
        img_x = np.pad(imgs_x[i], ((pad, pad), (pad, pad), (0, 0)), mode='reflect')
        img_y = np.pad(imgs_y[i], ((pad, pad), (pad, pad), (0, 0)), mode='reflect')

        windows_x = sliding_window_view(img_x, (patch_size, patch_size, 3))
        windows_y = sliding_window_view(img_y, (patch_size, patch_size, 3))

        H, W = windows_x.shape[:2]
        for _ in range(samples_per_image):
            y = np.random.randint(0, H)
            x = np.random.randint(0, W)
            patch_x = windows_x[y, x]
            target = img_y[y+pad, x+pad]  # центр патча

            hsv = rgb2hsv(patch_x)
            lab = rgb2lab(patch_x)
            gray = np.mean(patch_x, axis=2)
            sobel_edge = sobel(gray)

            feats = np.concatenate([
                patch_x.reshape(-1),
                hsv.reshape(-1),
                lab.reshape(-1),
                [np.mean(gray), np.std(gray), np.mean(sobel_edge)],
                [x / W, y / H]
            ])

            features.append(feats)
            targets.append(target)

    return np.array(features), np.array(targets)


In [5]:
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
def train_ridge_models(X, Y, alpha=1.0):
    models = []
    for c in range(3):
        model = XGBRegressor()
        model.fit(X, Y[:, c])
        models.append(model)
    return models


In [6]:
from skimage.color import rgb2hsv, rgb2lab
from skimage.filters import sobel

def predict_images_fast(imgs_x, models, patch_size=5):
    pad = patch_size // 2
    pred_images = np.zeros_like(imgs_x)

    for idx in range(imgs_x.shape[0]):
        img = np.pad(imgs_x[idx], ((pad, pad), (pad, pad), (0, 0)), mode='reflect').astype(float)
        pred = np.zeros((128 * 128, 3))
        count = 0
        for i in range(pad, 128 + pad):
            for j in range(pad, 128 + pad):
                patch = img[i - pad:i + pad + 1, j - pad:j + pad + 1, :]

                hsv = rgb2hsv(patch)
                lab = rgb2lab(patch)
                gray = np.mean(patch, axis=2)
                sobel_edge = sobel(gray)

                feat = np.concatenate([
                    patch.reshape(-1),
                    hsv.reshape(-1),
                    lab.reshape(-1),
                    [np.mean(gray), np.std(gray), np.mean(sobel_edge)],
                    [j / 128, i / 128]  # нормированные координаты
                ])

                pred[count] = [model.predict(feat.reshape(1, -1))[0] for model in models]
                count += 1

        pred_images[idx] = pred.reshape(128, 128, 3)

    return np.clip(pred_images, 0, 255).astype(np.uint8)


In [ ]:
# Параметры
patch_size = 5
samples_per_image = 3  # вместо ~16 тыс. пикселей
print("sampling")
# Сэмплирование и обучение
X_train, Y_train = extract_enhanced_features(train_images_x, train_images_y, patch_size, samples_per_image)
models = train_ridge_models(X_train, Y_train, alpha=1.0)
print("predicting")
# Предсказание
test_pred_y = predict_images_fast(test_images_x, models, patch_size=patch_size)


sampling


Extracting enhanced patches: 100%|██████████| 2245/2245 [00:05<00:00, 383.24it/s]


predicting


In [ ]:
test_pred_y.shape

In [ ]:
sub = pd.read_csv('/kaggle/input/tst-day-1/sample_submission.csv')
sub

In [ ]:
import pandas as pd
import numpy as np

# Предполагаем, что test_pred_y.shape == (2243, 128, 128, 3)

# 1. Меняем порядок каналов с RGB на BGR
test_pred_y_bgr = test_pred_y[..., ::-1]  # меняем местами каналы по последней оси

# 2. Векторизуем пиксели
flat_pixels = test_pred_y_bgr.reshape(test_pred_y_bgr.shape[0], -1)

# 3. Создаём DataFrame с колонками pixel0, pixel1, ...
pixel_cols = ['pixel' + str(i) for i in range(flat_pixels.shape[1])]
sub = pd.DataFrame(flat_pixels, columns=pixel_cols)

# 4. Добавляем колонку 'id' в начало с нужным диапазоном
ids = np.arange(0, 0 + test_pred_y.shape[0])
sub.insert(0, 'id', ids)

# Проверяем
print(sub.head())
print(sub.shape)  # (2243, 1 + 128*128*3)


In [ ]:
sub.to_csv('submission.csv', index=False)
#🙂
